In [1]:
from utils import *
from sklearn.cross_validation import train_test_split

trainset = sklearn.datasets.load_files(container_path = 'data', encoding = 'UTF-8')
trainset.data, trainset.target = separate_dataset(trainset,1.0)
print (trainset.target_names)
print (len(trainset.data))
print (len(trainset.target))

/home/jupyter/.local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


['negative', 'positive']
10662
10662


In [2]:
concat = ' '.join(trainset.data).split()
vocabulary_size = len(list(set(concat)))
data, count, dictionary, rev_dictionary = build_dataset(concat, vocabulary_size)
print('vocab from size: %d'%(vocabulary_size))
print('Most common words', count[4:10])
print('Sample data', data[:10], [rev_dictionary[i] for i in data[:10]])

vocab from size: 20332
Most common words [('film', 1453), ('movie', 1270), ('one', 727), ('like', 721), ('story', 477), ('much', 386)]
Sample data [8, 378, 24, 10158, 96, 118, 897, 5347, 1205, 5348] ['story', 'line', 'may', '127', 'years', 'old', 'el', 'crimen', 'del', 'padre']


In [3]:
GO = dictionary['GO']
PAD = dictionary['PAD']
EOS = dictionary['EOS']
UNK = dictionary['UNK']

In [4]:
import tensorflow as tf

In [27]:
def embed_seq(x, vocab_sz, embed_dim, name, zero_pad=True): 
    embedding = tf.get_variable(name, [vocab_sz, embed_dim]) 
    if zero_pad:
        embedding = tf.concat([tf.zeros([1, embed_dim]), embedding[1:, :]], 0) 
    x = tf.nn.embedding_lookup(embedding, x)
    return x

def position_encoding(inputs):
    T = tf.shape(inputs)[1]
    repr_dim = inputs.get_shape()[-1].value
    pos = tf.reshape(tf.range(0.0, tf.to_float(T), dtype=tf.float32), [-1, 1])
    i = np.arange(0, repr_dim, 2, np.float32)
    denom = np.reshape(np.power(10000.0, i / repr_dim), [1, -1])
    enc = tf.expand_dims(tf.concat([tf.sin(pos / denom), tf.cos(pos / denom)], 1), 0)
    return tf.tile(enc, [tf.shape(inputs)[0], 1, 1])

class Model:
    def __init__(self, dict_size, dimension_output, size_layer, num_layers):
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None])
        
        x = self.X
        
        with tf.variable_scope('embed'):
            x = embed_seq(x, dict_size, size_layer, 'word')
        x += position_encoding(x)
        
        def cells(size, reuse=False):
            return tf.nn.rnn_cell.LSTMCell(size,initializer=tf.orthogonal_initializer(),reuse=reuse)
        
        for n in range(num_layers):
            (out_fw, out_bw), (state_fw, state_bw) = tf.nn.bidirectional_dynamic_rnn(
                cell_fw = cells(size_layer // 2),
                cell_bw = cells(size_layer // 2),
                inputs = x,
                dtype = tf.float32,
                scope = 'bidirectional_rnn_left_%d'%(n))
            x = tf.concat((out_fw, out_bw), 2)
        
        max_pool = tf.reduce_max(x, axis = 1)
        avg_pool = tf.reduce_mean(x, axis = 1)
        print(max_pool, avg_pool)
        
        concat = tf.concat([max_pool, avg_pool], axis = -1)
        concat = tf.layers.dense(concat, size_layer * 2, activation = tf.nn.relu)
        self.logits = tf.layers.dense(concat, dimension_output)
        print(self.logits)
        
        self.cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits = self.logits, labels = self.Y
            )
        )
        self.optimizer = tf.train.AdamOptimizer(
            learning_rate = learning_rate
        ).minimize(self.cost)
        correct_pred = tf.equal(
            tf.argmax(self.logits, 1, output_type = tf.int32), self.Y
        )
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
        

In [17]:
size_layer = 128
num_layers = 2
embedded_size = 128
dimension_output = len(trainset.target_names)
learning_rate = 1e-3
maxlen = 50
batch_size = 128
dimension_output

2

In [28]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(len(dictionary), dimension_output, size_layer, num_layers)
sess.run(tf.global_variables_initializer())

Tensor("Max:0", shape=(?, 128), dtype=float32) Tensor("Mean:0", shape=(?, 128), dtype=float32)
Instructions for updating:
Use keras.layers.dense instead.
Tensor("dense_1/BiasAdd:0", shape=(?, 2), dtype=float32)
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [29]:
vectors = str_idx(trainset.data,dictionary,maxlen)
train_X, test_X, train_Y, test_Y = train_test_split(vectors, trainset.target,test_size = 0.2)

In [32]:
from tqdm import tqdm
import time

EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 3, 0, 0, 0

while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n' % (EPOCH))
        break

    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    pbar = tqdm(
        range(0, len(train_X), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        batch_x = train_X[i : min(i + batch_size, train_X.shape[0])]
        batch_y = train_Y[i : min(i + batch_size, train_X.shape[0])]
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
            },
        )
        assert not np.isnan(cost)
        train_loss += cost
        train_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    pbar = tqdm(range(0, len(test_X), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        batch_x = test_X[i : min(i + batch_size, test_X.shape[0])]
        batch_y = test_Y[i : min(i + batch_size, test_X.shape[0])]
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
            },
        )
        test_loss += cost
        test_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    train_loss /= len(train_X) / batch_size
    train_acc /= len(train_X) / batch_size
    test_loss /= len(test_X) / batch_size
    test_acc /= len(test_X) / batch_size

    if test_acc > CURRENT_ACC:
        print(
            'epoch: %d, pass acc: %f, current acc: %f'
            % (EPOCH, CURRENT_ACC, test_acc)
        )
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )
    EPOCH += 1


train minibatch loop:   1%|▏         | 1/67 [00:00<00:08,  7.91it/s, accuracy=0.445, cost=0.717]

epoch: 0, pass acc: 0.000000, current acc: 0.512771
time taken: 9.928096055984497
epoch: 0, training loss: 0.701227, training acc: 0.508563, valid loss: 0.715722, valid acc: 0.512771



train minibatch loop:   1%|▏         | 1/67 [00:00<00:08,  8.20it/s, accuracy=0.82, cost=0.426]

epoch: 1, pass acc: 0.512771, current acc: 0.747260
time taken: 9.40138840675354
epoch: 1, training loss: 0.671721, training acc: 0.551478, valid loss: 0.555007, valid acc: 0.747260



train minibatch loop:   1%|▏         | 1/67 [00:00<00:08,  7.99it/s, accuracy=0.898, cost=0.249]

...

time taken: 9.370667219161987
epoch: 5, training loss: 0.096641, training acc: 0.970836, valid loss: 0.818986, valid acc: 0.748898



train minibatch loop:   1%|▏         | 1/67 [00:00<00:08,  8.06it/s, accuracy=0.898, cost=0.207]

time taken: 9.3592209815979
epoch: 6, training loss: 0.109031, training acc: 0.963763, valid loss: 0.942395, valid acc: 0.701062



test minibatch loop: 100%|██████████| 17/17 [00:00<00:00, 17.31it/s, accuracy=0.776, cost=0.962]

time taken: 9.385342121124268
epoch: 7, training loss: 0.043385, training acc: 0.990034, valid loss: 1.374598, valid acc: 0.732953

break epoch:8

